In [1]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=2,                     # Number of cores per task (adjust if needed)
    processes=5,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:00:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)

In [2]:
from dask.distributed import Client

cluster.scale(jobs=10)
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:8787/status,
Dashboard: http://10.42.239.61:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:43357,Workers: 0
Dashboard: http://10.42.239.61:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import glob

In [16]:
dset_fol = '/home1/nalex2023/Datasets/IMERG/2013/2013_06/'
dset_fol2 = '/home1/nalex2023/Datasets/IMERG/2013/2013_12/'

all_files = glob.glob(dset_fol + '*.nc4')

all_files = all_files + glob.glob(dset_fol2 + '*.nc4')

outfol = '/home1/nalex2023/Datasets/IMERG/'


In [17]:
import xarray as xr
import pandas as pd

In [18]:

def prepro(ds):
    ds = ds['precipitation'].resample(time='1H').mean()
    return ds

dset = xr.open_mfdataset(all_files,parallel=True, combine='by_coords', preprocess=prepro)

In [19]:
dset

<xarray.Dataset> Size: 13GB
Dimensions:        (time: 8760, lon: 900, lat: 401)
Coordinates:
  * lat            (lat) float32 2kB -20.05 -19.95 -19.85 ... 19.75 19.85 19.95
  * lon            (lon) float32 4kB 90.05 90.15 90.25 ... 179.8 179.8 179.9
  * time           (time) object 70kB 2013-01-01 00:00:00 ... 2013-12-31 23:0...
Data variables:
    precipitation  (time, lon, lat) float32 13GB dask.array<chunksize=(1, 900, 401), meta=np.ndarray>

In [20]:
new_file = outfol + 'imerg_2013_maritime.nc'

dset.to_netcdf(new_file)

/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.91 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [9]:
years = range(2008, 2013)

for year in years:
    print(f'Processing year {year}')
    dset_fol = '/home1/nalex2023/Datasets/IMERG/' + str(year) + '/' + str(year) + '_06/'
    dset_fol2 = '/home1/nalex2023/Datasets/IMERG/' + str(year) + '/' + str(year) + '_12/'
    all_files = glob.glob(dset_fol + '*.nc4')
    all_files = all_files + glob.glob(dset_fol2 + '*.nc4')
    dset = xr.open_mfdataset(all_files,parallel=True, combine='by_coords', preprocess=prepro)
    new_file = outfol + 'imerg_' + str(year) + '_maritime.nc'
    dset.to_netcdf(new_file)
    # clear dask cache
    dset.close()
    print(f'Finished processing year {year}')

Processing year 2008


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.98 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Finished processing year 2008
Processing year 2009


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.91 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Finished processing year 2009
Processing year 2010


...

Finished processing year 2010
Processing year 2011


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.91 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Finished processing year 2011
Processing year 2012


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.98 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Finished processing year 2012


In [ ]:
client.close()